In [45]:
# import the necessary packages
import pandas as pd
import glob
import os
#from bs4 import BeautifulSoup as b
from pyproj import Proj 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
#import pytess
from shapely.geometry import Polygon
from matplotlib.lines import Line2D
from matplotlib.artist import Artist
from matplotlib.mlab import dist_point_to_segment
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import shapefile
import glob
from descartes import PolygonPatch
from shapely.ops import cascaded_union
from rtree import index
from shapely.geometry import Point
from shapely.ops import cascaded_union
from itertools import combinations
from shapely.ops import polygonize, unary_union
from prettytable import PrettyTable

In [2]:
surveys = pd.read_json("G:/drone/surveys/images.json")

In [3]:
def get_longitude(item):
    longitude =float(item[0]) + float(item[2][0:-1])/60 + float(item[3][0:-1])/3600
    return (longitude)

surveys['longitude'] = pd.np.NAN
surveys['latitude'] = pd.np.NAN
surveys.loc[ ~surveys['GPSLongitude'].isna(),'longitude']=surveys.loc[ ~surveys['GPSLongitude'].isna(),'GPSLongitude'].str.split(' ',expand=True).apply(get_longitude,axis=1)
surveys.loc[ ~surveys['GPSLatitude'].isna(),'latitude']=surveys.loc[ ~surveys['GPSLatitude'].isna(),'GPSLatitude'].str.split(' ',expand=True).apply(get_longitude,axis=1)*-1
surveys =surveys[~surveys.longitude.isna()]


In [4]:
myProj = Proj("+proj=utm +zone=49K, +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
surveys['easting'], surveys['northing'] = myProj(surveys['longitude'].values, surveys['latitude'].values)


In [5]:
surveys['rads']=np.deg2rad(surveys.FlightYawDegree)

In [6]:
surveys['BaseName']=surveys.SourceFile.apply(os.path.basename)
surveys['BaseName']=surveys['BaseName'].str[0:-4]
surveys['area']=surveys['BaseName'].str.split('_',expand=True)[0]

In [7]:
surveys.BaseName.str.contains('mb').max()
surveys

,About,AbsoluteAltitude,AlreadyApplied,Aperture,ApertureValue,BitsPerSample,CalibratedFocalLength,CalibratedOpticalCenterX,CalibratedOpticalCenterY,CamReverse,...,YCbCrSubSampling,YResolution,Yaw,longitude,latitude,easting,northing,rads,BaseName,area
0,DJI Meta Data,-20.00,False,6.3,6.3,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-67.9,113.940256,-21.961719,803673.709436,7.568494e+06,-1.185079,mbay_20181013T073611_2090,mbay
1,DJI Meta Data,-52.27,False,6.3,6.3,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-60.3,113.940283,-21.961694,803676.633396,7.568497e+06,-1.052434,mbay_20181015T073417_3634,mbay
2,DJI Meta Data,16.94,False,5.6,5.6,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-72.0,113.931967,-21.958275,802824.387700,7.568892e+06,-1.256637,mbay_20181014T082933_3548,mbay
3,DJI Meta Data,16.94,False,5.0,5.0,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,105.9,113.935675,-21.961672,803200.423030,7.568509e+06,1.848304,mbay_20181014T075841_3197,mbay
4,DJI Meta Data,17.04,False,5.6,5.6,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,106.2,113.934686,-21.962331,803096.826049,7.568438e+06,1.853540,mbay_20181014T075241_3061,mbay
5,DJI Meta Data,16.74,False,5.6,5.6,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-71.7,113.936450,-21.959544,803285.039121,7.568743e+06,-1.251401,mbay_20181014T082754_3509,mbay
6,DJI Meta Data,16.94,False,5.0,5.0,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,106.3,113.936744,-21.961039,803312.293053,7.568577e+06,1.855285,mbay_20181014T080454_3338,mbay
7,DJI Meta Data,16.94,False,5.6,5.6,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-71.9,113.935169,-21.961058,803149.479628,7.568578e+06,-1.254892,mbay_20181014T080016_3230,mbay
8,DJI Meta Data,16.94,False,5.6,5.6,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,-72.1,113.933831,-21.960681,803011.911317,7.568622e+06,-1.258382,mbay_20181014T080046_3242,mbay
9,DJI Meta Data,16.94,False,6.3,6.3,8,3666.666504,2432.0,1824.0,0,...,YCbCr4:2:2 (2 1),72,106.3,113.930678,-21.961194,802684.989926,7.568571e+06,1.855285,mbay_20181014T075111_3026,mbay


In [28]:
a=surveys.loc[surveys['easting'].idxmax()]
b=surveys.loc[surveys['easting'].idxmin()]
c=surveys.loc[surveys['northing'].idxmax()]
d=surveys.loc[surveys['northing'].idxmin()]


In [46]:
aa=a['easting'],a['northing']
bb=b['easting'],b['northing']
cc=c['easting'],c['northing']
dd=d['easting'],d['northing']

In [48]:
area = PrettyTable()

area.field_names = ["point name", "easting", "northing"]

area.add_row(["a",a['easting'],a['northing']])
area.add_row(["b", b['easting'],b['northing']])
area.add_row(["c",c['easting'],c['northing']])
area.add_row(["d", d['easting'],d['northing']])

print(area)

+------------+-------------------+-------------------+
| point name |      easting      |      northing     |
+------------+-------------------+-------------------+
|     a      | 815298.5082243532 | 7581971.833331805 |
|     b      | 796464.0017356293 | 7549873.186812805 |
|     c      | 815248.6158861786 |  7582065.19014718 |
|     d      | 796895.0828514984 | 7549796.684043875 |
+------------+-------------------+-------------------+


In [41]:
area=pd.DataFrame(joint,columns=['FileName','xxx','yyy','value'])

In [50]:
area

In [49]:
area.to_csv(r'C:\Users\BEA200\Desktop\turtle\python\table\area.csv')

AttributeError: to_csv